<h1 style="text-align: center;"><span style="color: #3366ff;"><strong>Business Location Analysis</strong></span></h1>

### Import necessary Libraries

In [14]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Credential was kept secret')


Your credentails:
CLIENT_ID: xxxxxxxxxxxxxxxxxx
CLIENT_SECRET:xxxxxxxxxxxxxxxx


##Read Configured Values

In [49]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

CLIENT_ID = config['DEFAULT']['CLIENT_ID']
CLIENT_SECRET = config['DEFAULT']['CLIENT_SECRET']
VERSION = '20180604'
LIMIT = 1000



#### Function definition for finding points of interests for a cordinate

In [4]:
POI_CATEGORY = ''
RADIUS = 5000

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_pointsofinterest_for_a_cordinate(latitude, longitude):   
    try:
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, str(latitude), str(longitude), VERSION, POI_CATEGORY, RADIUS, LIMIT);
        results = requests.get(url).json()  
        if(results['response'] == {}):
            if(results['meta']['errorType'] == 'quota_exceeded'):
                print('quota_exceeded')
            return pd.DataFrame(None)
            venues = results['response']['venues']
            dataframe = json_normalize(venues)

            filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
            dataframe_filtered = dataframe.loc[:, filtered_columns]

            # filter the category for each row
            dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

            # clean column names by keeping only last term
            dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
            #dataframe_filtered = dataframe_filtered[['categories', 'location.lat', 'location.lng', 'name']]
            dataframe_filtered = dataframe_filtered.loc[:, ['categories', 'lat', 'lng', 'name']]
            return dataframe_filtered
    except KeyError as e:                                                        
        return pd.DataFrame(None)
        ;        
    finally :
        ;
df = get_pointsofinterest_for_a_cordinate(19.132129, 72.812243)
df.head()

 
  
   
   
 
#### Search through the matrix of cordinates covering the central region of Mumbai City
##### Define geographical rectangle with LeftTop-RightDown cordinates covering Central Mumbai, generate all the cordinates in it with certain distance 

In [3]:

lefttop_latitude = 19.132129
lefttop_longitude = 72.812243
rightdown_latitude = 18.998439
rightdown_longitude = 72.935153
gap = 0.002
# --2

latitude = lefttop_latitude    
longitude = lefttop_longitude
cordinates = pd.DataFrame([[latitude, longitude]], columns = ['latitude', 'longitude'])

df_POI = get_pointsofinterest_for_a_cordinate(latitude, longitude)
latitude = latitude + gap
if df_POI.empty == False:
    roundno = 0
    while latitude > rightdown_latitude:
        longitude = lefttop_longitude
        while longitude < rightdown_longitude:
            df = get_pointsofinterest_for_a_cordinate(latitude, longitude)
            if df.empty == False:
                df_POI = df_POI.append(df)
                df_POI.drop_duplicates(keep=False,inplace=True)
                
                if (roundno%500 == 0) & (roundno != 0):
                    df_POI.to_csv('mumbai_002_'+ str(roundno) + '.csv')
                    df_POI=df_POI.head(1)
                    print('file created for : '+ str(roundno) + '  [', str(latitude), ', ', str(longitude), ']')
    
            roundno = roundno + 1
            longitude = longitude + gap        
        latitude = latitude - gap  
        
    df_POI.to_csv('mumbai_002_'+ str(roundno) + '.csv')
    print('file created for : '+ str(roundno))
else:
    print('Quota might be over')
        


All the coordinates were downloaded in multiple instances and saved in several files. In the following section those files were merged in to a single file.

In [ ]:

import os
import glob
import pandas as pd


#os.chdir("D:/study/coursera/project/merging trial/")        
         
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]


combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')


In [62]:
import pandas as pd
dataset = pd.read_csv('combined_csv.csv', header = 0)

#### Try DBScan Algorithm

In [ ]:
from sklearn.cluster import DBSCAN 
import pandas as pd
dataset = pd.read_csv('combined_csv.csv', header = 0)
X = dataset[['lat', 'lng']]
X.head()
epsilon = 0.3
minimumSamples = 7
db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(X)
labels = db.labels_
labels

#### Create Clusters using K means

In [37]:
from sklearn.cluster import KMeans 

X = dataset[['lat', 'lng']]
X.head()

k_means = KMeans(init="k-means++", n_clusters=1000, n_init=12)

k_means.fit(X)
labels = k_means.labels_

print(labels)
dataset["Labels"] = labels
# dataset.to_csv( "combined_csv_cluster.csv", index=False, encoding='utf-8-sig')


[450 400 400 ... 168 168 407]


#### Visualize the clusters on map

In [16]:

k_means_labels = k_means.labels_
k_means_labels
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers.shape

cluster_centers = pd.DataFrame(k_means_cluster_centers)

cluster_centers.columns = ['lat', 'lng']
import folium # plotting library
latitude = 19.076084
longitude = 72.877279

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='blue',
    popup='Mumbai',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(venues_map)


for i in range(len(cluster_centers)):    
    folium.features.CircleMarker(
#         [lat, lng],
        [cluster_centers.iloc[i]['lat'], cluster_centers.iloc[i]['lng']],
        radius=1,
        color='blue',
        popup=str(i),
        fill = True,
        fill_color='blue',
        fill_opacity=0.8
    ).add_to(venues_map)

# # display map
venues_map

NameError: name 'k_means' is not defined

Convert 'name' column to 'category' column and merge them by creating a separatte row.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from efficient_apriori import apriori
points = pd.read_csv('combined_csv_cluster.csv', header = 0)

points['categories'] = points['categories'] + ' ' + points['name']
points['categories'] = points['categories'].str.lower().str.replace('(', ' ').str.replace(')', ' ').str.replace('@', ' ').str.replace('-', ' ').str.replace('-', ' ').str.replace('  ', ' ').str.replace('&', ' ').str.replace('/', ' ').str.replace('\\', ' ').str.replace('\'', ' ').str.replace(',', ' ').str.replace('¤', ' ').str.replace('"', ' ').str.replace('é', 'e').str.replace('nan', '').str.replace('the', '').str.replace('and', '').str.replace('indian', '')

points.to_csv('D:/study/coursera/project/rough.csv')

points.columns
# print(dataset.head())
#le.fit_transform(dataset['categories'].astype(str))
#points['categories'] = points['categories'].str.lower().str.replace('/', ' ').str.replace('-', ' ').str.replace('  ', '')
newcols=[]
words = []
#words = pd.DataFrame(None)
#words = points[['Labels', 'categories']].head(0).copy()

#points = points.head(10)
for i in range(points['categories'].str.split(" ", n = 10, expand = True).shape[1]):
    newcols.append('categories' + '_'+ str(i) )
    points['categories' + '_'+ str(i)] = points['categories'].str.split(" ", n = 10, expand = True)[i]
#    print(points['categories'].str.replace('/', ' ').str.replace('-', ' ').str.replace('  ', '').str.split(" ", n = 10, expand = True)[i])
    
    #for row in points.rows:
    #for row in points.iterrows():
    for index, row in points.iterrows():
        #print(row['categories'])
        #print(row['categories' + '_'+ str(i)])
        if (row['categories' + '_'+ str(i)] != None) and (row['categories' + '_'+ str(i)] != 'None') and (row['categories' + '_'+ str(i)] != ' ') and (row['categories' + '_'+ str(i)] != ''):
            #print(row[['Labels','categories' + '_'+ str(i)]])
            words.append(row[['Labels','categories' + '_'+ str(i)]].values.tolist())
        #words.append(list(row['Labels'], row['categories' + '_'+ str(i)]))
        
wrd = pd.DataFrame(words).drop_duplicates()
#wrd['words'] = wrd['words'].apply(str).str.lower().str.replace('(', ' ').str.replace(')', ' ').str.replace('@', ' ').str.replace('-', ' ').str.replace('-', ' ').str.replace('&', ' ').str.replace('/', ' ').str.replace('\\', ' ').str.replace('\'', ' ').str.replace(',', ' ').str.replace('¤', ' ').str.replace('"', ' ').str.replace('é', 'e').str.replace('nan', '').str.replace('the', '').str.replace('and', '')
wrd.head()
wrd.columns = ['labels','words']
wrd = wrd[wrd['words'].str.len() > 1]

wrd = wrd[~wrd.words.str.contains('nan')]
wrd = wrd.drop_duplicates()
wrd.to_csv('words.csv')


In [69]:
wrd = pd.read_csv('words.csv')
wrd.head()
wrd[wrd['words'] == 'dessert'].head()

,Unnamed: 0,labels,words
196,269,640,dessert
214,290,199,dessert
364,492,271,dessert
386,525,197,dessert
675,928,45,dessert


#### Find cluster not having dessert shops

In [79]:
wrd = pd.read_csv('words.csv')
wrd.head()

# venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

k_means_labels = k_means.labels_
k_means_labels
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers.shape

cluster_centers = pd.DataFrame(k_means_cluster_centers)

cluster_centers.columns = ['center_lat', 'center_lng']

cluster_centers = cluster_centers.reset_index()
cluster_centers['labels'] = cluster_centers.index + 0

wrd = pd.merge(wrd, cluster_centers, on = 'labels', how = 'outer')

dessert = wrd[wrd['words'] == 'dessert'][['center_lat', 'center_lng']].drop_duplicates()

# wrd.head()
import folium # plotting library
latitude = 19.076084
longitude = 72.877279
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='blue',
    popup='Mumbai',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(venues_map)


for i in range(len(cluster_centers)): 
    folium.features.CircleMarker(
#         [lat, lng],
        [cluster_centers.iloc[i]['center_lat'], cluster_centers.iloc[i]['center_lng']],
        radius=1,
        color='blue',
        popup=str(i),
        fill = True,
        fill_color='blue',
        fill_opacity=0.8
    ).add_to(venues_map)

for i in range(len(dessert)): 
    folium.features.CircleMarker(
#         [lat, lng],
        [dessert.iloc[i]['center_lat'], dessert.iloc[i]['center_lng']],
        radius=2,
        color='red',
        popup=str(i),
        fill = True,
        fill_color='yellow',
        fill_opacity=0.8
    ).add_to(venues_map)    
    
# # display map
venues_map

In [85]:
dessert = wrd[wrd['words'] == 'dessert'][['center_lat', 'center_lng']].drop_duplicates()

common = wrd.merge(dessert,on=['center_lat', 'center_lng'])
# print(common)
non_desert = wrd[(~wrd.center_lat.isin(common.center_lat))&(~wrd.center_lng.isin(common.center_lng))][['center_lat', 'center_lng']].drop_duplicates()
non_desert

,center_lat,center_lng
0,19.051154,72.893990
87,19.097033,72.850748
167,19.125291,72.921495
274,19.072219,72.866539
394,19.042699,72.862248
504,18.961169,72.931108
625,19.060156,72.853508
727,19.061127,72.893487
817,19.097900,72.902340
969,19.073118,72.823106


#### Extended Analysis with Apriori

In [ ]:
wrd['words'] = wrd['words'].apply(str).str.lower()

for i in range(0,wrd['labels'].max()):
    datalst = wrd[wrd['labels'] == i]['words'].tolist()
    transactions2.append(tuple(datalst))
    

wrd['words'] = wrd['words'].apply(str).str.lower()

for i in range(0,points['Labels'].max()):
    datalst = points[points['Labels'] == i]['categories'].apply(str).tolist()
    transactions2.append(tuple(datalst))    
    datalst = points[points['Labels'] == i]['name'].apply(str).tolist()
    transactions2.append(tuple(datalst))
    
from efficient_apriori import apriori

itemsets, rules = apriori(transactions2, min_support=0.08, min_confidence=0.4)

<a id="item1"></a>

<a id="item2"></a>

<a id="item3"></a>

<a id="item4"></a>

<a id="item5"></a>

<a id="item6"></a>